In [188]:
import pandas as pd
%matplotlib inline
from matplotlib import pyplot as plt
import numpy as np
from scipy.stats import ttest_ind
import math

In [189]:
'''READS IN TEAM STATS FOR AND CORRECTS NAMING ERRORS'''
team_stats1314 = pd.read_csv('./preprocessed/team_stats1314.csv', sep=',')
team_stats1415 = pd.read_csv('./preprocessed/team_stats1415.csv', sep=',')
team_stats1516 = pd.read_csv('./preprocessed/team_stats1516.csv', sep=',')
team_stats1617 = pd.read_csv('./preprocessed/team_stats1617.csv', sep=',')
team_stats1718 = pd.read_csv('./preprocessed/team_stats1718.csv', sep=',')

dfs = [team_stats1314, team_stats1415, team_stats1516, team_stats1617, team_stats1718]

for df in dfs:
    df['Team'] = df.Team.str.replace(' ', '_')
    df['Team'] = df['Team'].str[:-1]

team_stats1415.head()

,Unnamed: 0,Rk,Team,G,MP,FG,FGA,FG%,3P,3PA,...,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS,WIN%
0,0,1,Golden_State_Warriors,82,240.6,41.6,87.0,0.478,10.8,27.0,...,10.4,34.3,44.7,27.4,9.3,6.0,14.5,19.9,110.0,81.7
1,1,4,LA_Clippers,82,240.6,39.4,83.3,0.473,10.1,26.9,...,9.6,33.1,42.6,24.8,7.8,5.0,12.3,21.3,106.7,68.3
2,2,9,Dallas_Mavericks,82,242.4,39.7,85.8,0.463,8.9,25.4,...,10.5,31.8,42.3,22.5,8.1,4.5,13.0,20.0,105.2,61.0
3,3,11,Toronto_Raptors,82,242.1,37.9,83.3,0.455,8.9,25.1,...,10.7,30.8,41.5,20.7,7.5,4.4,12.9,20.9,104.0,59.8
4,4,3,Houston_Rockets,82,241.5,37.0,83.3,0.444,11.4,32.7,...,11.7,32.0,43.7,22.2,9.5,5.0,16.7,22.0,103.9,68.3


In [190]:
'''READS IN TEAM STATS AGAINST'''
opp_stats1314 = pd.read_csv('./preprocessed/opp_stats1314.csv', sep=',')
opp_stats1415 = pd.read_csv('./preprocessed/opp_stats1415.csv', sep=',')
opp_stats1516 = pd.read_csv('./preprocessed/opp_stats1516.csv', sep=',')
opp_stats1617 = pd.read_csv('./preprocessed/opp_stats1617.csv', sep=',')
opp_stats1718 = pd.read_csv('./preprocessed/opp_stats1718.csv', sep=',')

opp_arr = [opp_stats1314, opp_stats1415, opp_stats1516, opp_stats1617, opp_stats1718]

for df in opp_arr:
    df['Team'] = df['Team'].str[:-1]

In [191]:
'''READS IN SERIES INFORMATION CSV FILES'''
series_in1314 = pd.read_csv('./Playoff_Matchups/Playoff_13-14.csv', sep=',')
series_in1415 = pd.read_csv('./Playoff_Matchups/Playoff_14-15.csv', sep=',')
series_in1516 = pd.read_csv('./Playoff_Matchups/Playoff_15-16.csv', sep=',')
series_in1617 = pd.read_csv('./Playoff_Matchups/Playoff_16-17.csv', sep=',')
series_in1415.head()

,TeamA,TeamB,Winner
0,Atlanta_Hawks,Brooklyn_Nets,0
1,Chicago_Bulls,Milwaukee_Bucks,0
2,Cleveland_Cavaliers,Boston_Celtics,0
3,Golden_State_Warriors,New_Orleans_Pelicans,0
4,Portland_Trail_Blazers,Memphis_Grizzlies,1


In [192]:
def make_series(tm, series, df):   
    if tm == 'a':
        a = series['TeamA'].tolist()
    else:
        a = series['TeamB'].tolist()
            
    team = pd.DataFrame()
    
    for x in a:
        to_concat = df[df['Team'] == x]
        frames = [team, to_concat]
        team = pd.concat(frames)
    
    return team  

In [193]:
'''THIS MAKES THE FOR AND OPPOSING TABLES FOR THE PLAYOFFS TEAM A'''
a = 'a'
team_a1314 = make_series(a, series_in1314, team_stats1314)
team_a_opp1314 = make_series(a, series_in1314, opp_stats1314)
team_a1415 = make_series(a, series_in1415, team_stats1415)
team_a_opp1415 = make_series(a, series_in1415, opp_stats1415)
team_a1516 = make_series(a, series_in1516, team_stats1516)
team_a_opp1516 = make_series(a, series_in1516, opp_stats1516)
team_a1617 = make_series(a, series_in1617, team_stats1617)
team_a_opp1617 = make_series(a, series_in1617, opp_stats1617)

team_a_opp1617.head()

,Unnamed: 0,Team,G,MP,FG,FGA,FG%,3P,3PA,3P.1,...,FT%,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS
10,10,Boston_Celtics,82,240.9,38.6,85.8,0.450,9.0,27.0,0.332,...,0.769,10.8,33.8,44.5,22.6,7.6,5.2,13.9,20.3,105.4
13,13,Washington_Wizards,82,242.1,39.7,85.1,0.466,9.9,27.1,0.364,...,0.751,10.6,32.5,43.0,22.8,7.4,4.6,15.3,19.8,107.4
4,4,Toronto_Raptors,82,241.2,37.2,82.9,0.449,9.7,27.3,0.354,...,0.780,10.1,31.9,42.1,21.8,6.7,4.8,14.6,20.3,102.6
12,12,Cleveland_Cavaliers,82,242.4,40.6,88.8,0.458,10.4,28.7,0.361,...,0.773,11.0,33.2,44.2,24.7,7.3,4.3,12.6,20.6,107.2
7,7,Golden_State_Warriors,82,241.2,38.8,89.2,0.435,9.0,27.8,0.324,...,0.761,11.7,31.8,43.5,22.7,8.6,3.8,15.5,19.4,104.3


In [194]:
'''THIS MAKES THE FOR AND OPPOSING TABLES FOR THE PLAYOFFS TEAM B'''
b = 'b'
team_b1314 = make_series(b, series_in1314, team_stats1314)
team_b_opp1314 = make_series(b, series_in1314, opp_stats1314)
team_b1415 = make_series(b, series_in1415, team_stats1415)
team_b_opp1415 = make_series(b, series_in1415, opp_stats1415)
team_b1516 = make_series(b, series_in1516, team_stats1516)
team_b_opp1516 = make_series(b, series_in1516, opp_stats1516)
team_b1617 = make_series(b, series_in1617, team_stats1617)
team_b_opp1617 = make_series(b, series_in1617, opp_stats1617)

team_b1617.head()

,Unnamed: 0,Rk,Team,G,MP,FG,FGA,FG%,3P,3PA,...,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS,WIN%
13,13,15,Chicago_Bulls,82,241.2,38.6,87.1,0.444,7.6,22.3,...,12.2,34.1,46.3,22.6,7.8,4.8,13.6,17.7,102.9,50.0
12,12,11,Atlanta_Hawks,82,242.4,38.1,84.4,0.451,8.9,26.1,...,10.3,34.1,44.3,23.6,8.2,4.8,15.8,18.2,103.2,52.4
11,11,13,Milwaukee_Bucks,82,241.2,38.8,81.9,0.474,8.8,23.7,...,8.8,31.6,40.4,24.2,8.1,5.3,14.0,20.2,103.6,51.2
10,10,13,Indiana_Pacers,82,242.1,39.3,84.5,0.465,8.6,23.0,...,9.0,33.0,42.0,22.5,8.2,5.0,13.8,19.5,105.1,51.2
6,6,15,Portland_Trail_Blazers,82,243.0,39.5,86.1,0.459,10.4,27.7,...,10.1,33.5,43.7,21.1,7.0,5.0,13.7,21.2,107.9,50.0


In [195]:
'''THIS MAKES THE WINNER DATAFRAME'''
winner1314 = series_in1314['Winner']
winner1415 = series_in1415['Winner']
winner1516 = series_in1516['Winner']
winner1617 = series_in1617['Winner']

win1314 = pd.DataFrame(winner1314, columns = ["Winner"])
win1415 = pd.DataFrame(winner1415, columns = ["Winner"])
win1516 = pd.DataFrame(winner1516, columns = ["Winner"])
win1617 = pd.DataFrame(winner1617, columns = ["Winner"])

win1314.head()

,Winner
0,0
1,1
2,1
3,0
4,0


In [196]:
l = [team_a_opp1314, team_a_opp1415, team_a_opp1516, team_a_opp1617, team_b_opp1314, team_b_opp1415, team_b_opp1516, team_b_opp1617]

for j in l:
    del j['Unnamed: 0']
    del j['Team']
    del j['G']
    del j['MP']

team_a_opp1314.columns

Index(['FG', 'FGA', 'FG%', '3P', '3PA', '3P%', '2P', '2PA', '2P%', 'FT', 'FTA',
       'FT%', 'ORB', 'DRB', 'TRB', 'AST', 'STL', 'BLK', 'TOV', 'PF', 'PTS'],
      dtype='object')

In [197]:
opps = [team_a_opp1314, team_a_opp1415, team_a_opp1516, team_a_opp1617, team_b_opp1314, team_b_opp1415, team_b_opp1516, team_b_opp1617]

for i in opps:
    i.columns = ['FG_opp', 'FGA_opp', 'FG%_opp', '3P_opp', '3PA_opp', '3P%_opp', '2P_opp', '2PA_opp', '2P%_opp', 'FT_opp', 'FTA_opp',
       'FT%_opp', 'ORB_opp', 'DRB_opp', 'TRB_opp', 'AST_opp', 'STL_opp', 'BLK_opp', 'TOV_opp', 'PF_opp', 'PTS_opp']


In [198]:
opp_cols = team_a_opp1314.columns
opp_cols

# team_b1314[col] = list(team_b_opp1314[col])

for col in opp_cols:
    team_a1314[col] = list(team_a_opp1314[col])
    team_a1415[col] = list(team_a_opp1415[col])
    team_a1516[col] = list(team_a_opp1516[col])
    team_a1617[col] = list(team_a_opp1617[col])
    team_b1314[col] = list(team_b_opp1314[col])
    team_b1415[col] = list(team_b_opp1415[col])
    team_b1516[col] = list(team_b_opp1516[col])
    team_b1617[col] = list(team_b_opp1617[col])

team_a1617.head()

,Unnamed: 0,Rk,Team,G,MP,FG,FGA,FG%,3P,3PA,...,FT%_opp,ORB_opp,DRB_opp,TRB_opp,AST_opp,STL_opp,BLK_opp,TOV_opp,PF_opp,PTS_opp
5,5,4,Boston_Celtics,82,240.9,38.6,85.1,0.454,12.0,33.4,...,0.769,10.8,33.8,44.5,22.6,7.6,5.2,13.9,20.3,105.4
3,3,9,Washington_Wizards,82,242.1,41.3,87.0,0.475,9.2,24.8,...,0.751,10.6,32.5,43.0,22.8,7.4,4.6,15.3,19.8,107.4
7,7,5,Toronto_Raptors,82,241.2,39.2,84.4,0.464,8.8,24.3,...,0.780,10.1,31.9,42.1,21.8,6.7,4.8,14.6,20.3,102.6
2,2,5,Cleveland_Cavaliers,82,242.4,39.9,84.9,0.470,13.0,33.9,...,0.773,11.0,33.2,44.2,24.7,7.3,4.3,12.6,20.6,107.2
0,0,1,Golden_State_Warriors,82,241.2,43.1,87.1,0.495,12.0,31.2,...,0.761,11.7,31.8,43.5,22.7,8.6,3.8,15.5,19.4,104.3


In [199]:
team_a = [team_a1314, team_a1415, team_a1516, team_a1617]
team_b = [team_b1314, team_b1415, team_b1516, team_b1617]
arr = team_a + team_b

for j in arr:
    del j['Unnamed: 0']
    del j['Team']
#     print(j.dtypes)
#     print(j)

In [200]:
def find_diff(col, cols, year_a, year_b, df):
    col_name = cols[col]
    end = []
    diff_end = []
    diff_arr = list(year_a[col_name])
    diff_arr2 = list(year_b[col_name])
    for x in range(len(diff_arr)):
        diff = diff_arr[x] - diff_arr2[x]
        end.append(diff)
    df[col] = end

In [201]:
cols = list(team_a1314.columns)
# print(len(cols))

df1314 = pd.DataFrame()
df1415 = pd.DataFrame()
df1516 = pd.DataFrame()
df1617 = pd.DataFrame()

years = [df1314, df1415, df1516, df1617]

for year in range(len(years)):
    for col in range(len(cols)):
        find_diff(col, cols, team_a[year], team_b[year], years[year])
df1516.shape

(15, 46)

In [202]:
df1314.to_csv('./preprocessed/final/diff1314.csv')
df1415.to_csv('./preprocessed/final/diff1415.csv')
df1516.to_csv('./preprocessed/final/diff1516.csv')
df1617.to_csv('./preprocessed/final/diff1617.csv')

In [206]:
def normalize_df(df):
    cols = df.columns
    for col in cols:
        df = df.apply(lambda x: (x - x.min()) / (x.max() - x.min()))
    return df

In [207]:
df_norm1314 = normalize_df(df1314)
df_norm1415 = normalize_df(df1415)
df_norm1516 = normalize_df(df1516)
df_norm1617 = normalize_df(df1617)

In [208]:
df_norm1314.to_csv('./preprocessed/final/diff_norm1314.csv')
df_norm1415.to_csv('./preprocessed/final/diff_norm1415.csv')
df_norm1516.to_csv('./preprocessed/final/diff_norm1516.csv')
df_norm1617.to_csv('./preprocessed/final/diff_norm1617.csv')

In [ ]:
# '''WRITES TEAM A STATS TO CSV FILES'''
# team_a1314.to_csv('./preprocessed/final/playoffs_for_a_1314.csv')
# team_a1415.to_csv('./preprocessed/final/playoffs_for_a_1415.csv')
# team_a1516.to_csv('./preprocessed/final/playoffs_for_a_1516.csv')
# team_a1617.to_csv('./preprocessed/final/playoffs_for_a_1617.csv')

# team_a_opp1314.to_csv('./preprocessed/final/playoffs_opp_a_1314.csv')
# team_a_opp1415.to_csv('./preprocessed/final/playoffs_opp_a_1415.csv')
# team_a_opp1516.to_csv('./preprocessed/final/playoffs_opp_a_1516.csv')
# team_a_opp1617.to_csv('./preprocessed/final/playoffs_opp_a_1617.csv')

In [ ]:
# '''WRITES TEAM B STATS TO CSV FILES'''
# team_b1314.to_csv('./preprocessed/final/playoffs_for_b_1314.csv')
# team_b1415.to_csv('./preprocessed/final/playoffs_for_b_1415.csv')
# team_b1516.to_csv('./preprocessed/final/playoffs_for_b_1516.csv')
# team_b1617.to_csv('./preprocessed/final/playoffs_for_b_1617.csv')

# team_b_opp1314.to_csv('./preprocessed/final/playoffs_opp_b_1314.csv')
# team_b_opp1415.to_csv('./preprocessed/final/playoffs_opp_b_1415.csv')
# team_b_opp1516.to_csv('./preprocessed/final/playoffs_opp_b_1516.csv')
# team_b_opp1617.to_csv('./preprocessed/final/playoffs_opp_b_1617.csv')

In [ ]:
# '''WRITES WINNER DATAFRAME TO CSV FILES'''
#win1314.to_csv('./preprocessed/final/winner1314.csv')
#win1415.to_csv('./preprocessed/final/winner1415.csv')
#win1516.to_csv('./preprocessed/final/winner1516.csv')
#win1617.to_csv('./preprocessed/final/winner1617.csv')

In [ ]:
#team_stats1718 drop unnamed, team, g, mp
#del team_stats1718['Unnamed: 0']
#del team_stats1718['Team']
#del team_stats1718['G']
#del team_stats1718['MP']

In [ ]:
#opp_stats1718
#del opp_stats1718['Unnamed: 0']
#del opp_stats1718['Team']
#del opp_stats1718['G']
#del opp_stats1718['MP']

In [79]:
team_stats1718.head()

,Rk,FG,FGA,FG%,3P,3PA,3P%,2P,2PA,2P%,...,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS,WIN%
0,3,42.8,85.1,0.503,11.3,28.9,0.391,31.5,56.2,0.560,...,8.4,35.1,43.5,29.3,8.0,7.5,15.5,19.6,113.5,70.7
1,1,38.7,84.2,0.460,15.3,42.3,0.362,23.4,41.9,0.558,...,9.0,34.5,43.5,21.5,8.5,4.8,13.8,19.5,112.4,79.3
2,8,42.7,88.3,0.483,10.2,28.2,0.362,32.5,60.1,0.540,...,8.7,35.7,44.3,26.8,8.0,5.9,14.9,19.1,111.7,58.5
3,2,41.3,87.4,0.472,11.8,33.0,0.358,29.5,54.5,0.541,...,9.8,34.2,44.0,24.3,7.6,6.1,13.4,21.8,111.7,72.0
4,6,40.4,84.8,0.476,12.0,32.1,0.372,28.4,52.6,0.540,...,8.5,33.7,42.1,23.4,7.1,3.8,13.7,18.6,110.9,61.0


In [80]:
opp_stats1718.head()

,FG,FGA,FG%,3P,3PA,3P%,2P,2PA,2P%,FT,...,FT%,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS
0,37.3,83.0,0.449,9.5,26.0,0.365,27.8,57.0,0.488,15.8,...,0.771,8.7,33.0,41.6,20.0,8.3,4.6,14.9,20.5,99.8
1,38.2,84.3,0.453,9.1,26.3,0.348,29.0,58.0,0.501,14.3,...,0.759,9.2,33.3,42.5,21.8,7.6,3.9,14.1,19.8,99.8
2,37.4,85.0,0.440,9.4,27.7,0.339,28.0,57.4,0.488,16.3,...,0.763,9.6,34.2,43.8,21.3,7.2,4.4,14.1,19.2,100.4
3,37.7,83.8,0.450,9.6,26.6,0.360,28.1,57.2,0.491,18.0,...,0.783,9.1,34.0,43.1,21.0,7.5,4.6,14.1,19.4,102.9
4,38.4,85.9,0.447,9.7,26.6,0.364,28.7,59.3,0.484,16.6,...,0.755,9.3,33.6,42.9,20.2,7.3,5.1,12.6,19.1,103.0
